<center style="padding: 1em 2em;  margin: 1em 0;">
    <h1 style="font-size: 2.2em; margin: 0 0 0.5em 0; color:rgb(52, 72, 92);">
        «Введение в Deploy. Streamlit.»
    </h1>
    <div style="font-size: 1.4em; color:rgb(52, 72, 92); font-weight: 600;">
        Зуев Гордей
    </div>
</center>

## **Сегодня пройдемся по этому плану:**

1. **Как подготовить и сохранить ML-модель** через `Pickle` для использования в production
2. **Как создать интерактивное веб-приложение** с помощью `Streamlit`
3. **Как задеплоить приложение** в `Streamlit Cloud` и поделиться им с миром

---

## **🔹 Часть 1: Подготовка модели и глубокая работа с Pickle**

### **Шаг 1: Предобработка данных и обучение модели**

<div style="padding: 12px 16px; margin: 16px 0; border-radius: 6px; border-left: 4px solid #3b82f6; background-color: #eff6ff; color: #1e40af;">
<strong>ℹ️ Важно!</strong><br>Для production мы должны применить те же трансформации к новым данным, что и при обучении. Это гарантирует корректность предсказаний.
</div>


**Датасет:** Предсказание оттока клиентов (`Churn Prediction`)

**Ссылка на датасет:** `https://raw.githubusercontent.com/Murcha1990/datasets/refs/heads/main/churn_cliens_demo.csv`

**Задача:** Предсказать, уйдет ли клиент из компании (бинарная классификация)

### **Шаг 1.1: Загрузка данных и библиотек**

In [ ]:
# Основные библиотеки
import pandas as pd  # работа с таблицами
import matplotlib.pyplot as plt  # базовые визуализации
import seaborn as sns  # красивые графики поверх matplotlib

# ML библиотеки
from sklearn.model_selection import train_test_split  # разделяем данные на train/test

from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    roc_auc_score,
    roc_curve,
)  # базовые метрики качества
from catboost import CatBoostClassifier  # модель, которая умеет работать с категориальными строками

# Для сохранения модели
import joblib  # показываем альтернативный способ сериализации

In [ ]:
# Загрузка данных
url = 'https://raw.githubusercontent.com/Murcha1990/datasets/refs/heads/main/churn_cliens_demo.csv'
df = pd.read_csv(url)

df.to_csv("churn_cliens_demo.csv", index=False)  # сохраняем на диск, чтобы использовать в Streamlit

In [ ]:
print(f"Размер датасета: {df.shape}")

In [ ]:
df.sample(10)

**Быстрый обзор данных:**

In [ ]:
print(f"\nКоличество строк: {len(df)}")

In [ ]:
print(f"Количество столбцов: {len(df.columns)}")

In [ ]:
print(f"\nСтолбцы:\n{df.columns.tolist()}")

In [ ]:
print(f"\nПропущенные значения:\n{df.isnull().sum().sum()} всего")

In [ ]:
print(f"\nТипы данных:\n{df.dtypes.value_counts()}")

### **Целевая переменная**

In [ ]:
# Проверка баланса классов
print("Распределение целевой переменной (churn):")
print(df['churn'].value_counts())

In [ ]:
print(f"\nПроцентное соотношение:\n{df['churn'].value_counts(normalize=True) * 100}")

In [ ]:
# Визуализация
plt.figure(figsize=(10, 5))

# Преобразуем boolean в int для корректного отображения
churn_counts = df['churn'].astype(int).value_counts()
churn_counts.plot(kind='bar', color=['#2ecc71', '#e74c3c'])

plt.title('Распределение целевой переменной', fontsize=14)
plt.xlabel('Churn', fontsize=12)
plt.ylabel('Количество клиентов', fontsize=12)
plt.xticks(rotation=0)
plt.grid(axis='y', alpha=0.3)

plt.show()

In [ ]:
# Визуализация
plt.figure(figsize=(10, 5))

churn_counts.plot(kind='pie', autopct='%1.1f%%', colors=['#2ecc71', '#e74c3c'], startangle=90, labels=['Остался', 'Ушёл'])
plt.title('Доля клиентов', fontsize=14)
plt.ylabel('')

plt.show()

**Вопрос к аудитории:** Что такое дисбаланс классов? Почему это важно?

**Ответ:** Если одного класса значительно больше другого, модель может просто "научиться" предсказывать самый частый класс и иметь высокую точность, но быть бесполезной.

## **Шаг 1.2: Подготовка данных для модели**

Для того чтобы модель работала в `production`, нам нужно:
1. **Сохранить все трансформации** (кодирование, нормализация)
2. **Применять те же самые трансформации** к новым данным
3. **Обеспечить консистентность** между обучением и предсказанием

**Вопрос:** Что произойдёт, если мы обучим модель на нормализованных данных, а в production будем подавать ненормализованные?

**Ответ:** Модель будет давать некорректные предсказания!

In [ ]:
# Создаем копию данных для обработки
df_processed = df.copy()

# Проверим типы данных
print("Информация о типах данных:")
print(df_processed.info())

In [ ]:
# Проверим пропуски в данных
print("Проверка пропусков:")
missing = df_processed.isnull().sum()
missing

In [ ]:
if missing.sum() > 0:
    print(missing[missing > 0])
    
    # Создаем словарь с средними значениями для числовых столбцов
    numeric_means = df_processed.select_dtypes(include=['int64', 'float64']).mean()
    
    # Создаем словарь с модами для категориальных столбцов
    categorical_modes = df_processed.select_dtypes(exclude=['int64', 'float64']).mode().iloc[0]
    
    # Объединяем словари
    fill_values = {**numeric_means, **categorical_modes}
    
    # Заполняем пропуски за одну операцию
    df_processed = df_processed.fillna(fill_values)
else:
    print("✅ Пропусков нет!")

print(f"\nПропуски после обработки: {df_processed.isnull().sum().sum()}")

### Подготовка категориальных признаков для CatBoost

<div style="padding: 12px 16px; margin: 16px 0; border-radius: 6px; border-left: 4px solid #3b82f6; background-color: #eff6ff; color: #1e40af;">
<strong>ℹ️ Важно!</strong><br>CatBoost работает с категориальными признаками напрямую, поэтому нам не нужно их кодировать через LabelEncoder. Достаточно преобразовать их в строковый тип.
</div>


In [ ]:
# Обработка категориальных признаков
# Найдем все категориальные колонки (тип object)

categorical_columns = df_processed.select_dtypes(include=['object']).columns.tolist()
categorical_columns

In [ ]:
df_processed["internationalplan"].unique()

In [ ]:
# Исключаем целевую переменную 'churn'
categorical_columns = list(set(categorical_columns) - {'churn'})

print(f"Категориальные признаки для кодирования: {categorical_columns}")
print(f"Количество: {len(categorical_columns)}")

In [ ]:
# Преобразуем категориальные признаки в строки для CatBoost
df_processed[categorical_columns] = df_processed[categorical_columns].astype(str)

# Анализ категориальных признаков в одном цикле
for column in categorical_columns:
    unique_vals = df_processed[column].unique()
    n_unique = len(unique_vals)
    
    print(f"\n{column}:")
    show_count = 5 if n_unique <= 5 else 10
    print(f"  Классы (первые {show_count}): {unique_vals[:show_count].tolist()}")
    print(f"  Всего классов: {n_unique}")

print(f"\n✅ Подготовлено {len(categorical_columns)} категориальных признаков для CatBoost")

In [ ]:
# Подготовка признаков и целевой переменной
X = df_processed.drop(['churn'], axis=1)
y = df_processed['churn']

# Преобразуем y к числовому типу (из bool в int)
y = y.astype(int)

print(f"Форма признаков X: {X.shape}")
print(f"Форма целевой переменной y: {y.shape}")
print(f"Тип y: {y.dtype}")
print(f"\nПризнаки для модели ({len(X.columns)}):")
for i, col in enumerate(X.columns, 1):
    print(f"  {i:2}. {col}")


### **Разделение данных**

**Вопрос:** Зачем нужен параметр `stratify`?

**Ответ:** Он обеспечивает одинаковое соотношение классов в train и test!


In [ ]:
# Разделение на train и test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42,
    stratify=y  # Важно для сбалансированного разделения!
)

print(f"Train set: {X_train.shape}")
print(f"Test set: {X_test.shape}")
print(f"\nРаспределение классов в train:")
print(y_train.value_counts())
print(f"\nРаспределение классов в test:")
print(y_test.value_counts())


In [ ]:
# Проверим, что пропорции сохранились
train_churn_rate = y_train.sum() / len(y_train) * 100
test_churn_rate = y_test.sum() / len(y_test) * 100
print(f"\n✅ Процент оттока в train: {train_churn_rate:.1f}%")
print(f"✅ Процент оттока в test: {test_churn_rate:.1f}%")

In [ ]:
# CatBoost работает с категориальными признаками "из коробки"
# Поэтому вместо нормализации подготовим список cat_features
cat_features = [X.columns.get_loc(col) for col in categorical_columns if col in X.columns]

print("✅ Сформирован список cat_features для CatBoost")
print(f"Категориальных признаков: {len(cat_features)}")
print(f"Индексы cat_features: {cat_features}")


In [ ]:
print("Категориальные признаки, которые передадим в CatBoost:")
for col in categorical_columns:
    print(f" - {col}")

### **Шаг 1.3: Обучение финальной модели**

#### 💡 Обсуждение: Выбор модели для production

**Для production мы выбрали CatBoost, потому что:**
- ✅ Нативно работает с категориальными признаками (`cat_features`)
- ✅ Автоматически обрабатывает пропуски и категориальные значения
- ✅ Даёт важности признаков и поддерживает explainability-инструменты
- ✅ Быстро обучается и хорошо переносится в production (есть бинарные модели)

**Альтернативы:**
- `Logistic Regression` (проще, но требует сильной подготовки признаков)
- `Random Forest`(привычный бэйзлайн, но проигрывает на категориальных фичах)

In [ ]:
# Обучение модели
print("🚀 Начинаем обучение CatBoost...\n")

import time
start_time = time.time()

model = CatBoostClassifier(
    iterations=100,
    depth=2,
    learning_rate=0.1,
    loss_function='Logloss',
    eval_metric='AUC',
    cat_features=cat_features,
    random_seed=42,
    verbose=False
)

model.fit(
    X_train,
    y_train,
    eval_set=(X_test, y_test),
    use_best_model=True,
    verbose=False
)

training_time = time.time() - start_time
best_auc = model.get_best_score().get('validation', {}).get('AUC')
print(f"\n✅ CatBoost обучен за {training_time:.2f} секунд!")
if best_auc is not None:
    print(f"📈 Лучший AUC на валидации: {best_auc:.4f}")


### **Оценка модели**

In [ ]:
# Делаем предсказания (CatBoost принимает исходные признаки)
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Вычисляем метрики
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print("=" * 60)
print("РЕЗУЛЬТАТЫ МОДЕЛИ")
print("=" * 60)
print(f"\n📊 Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"📊 ROC-AUC: {roc_auc:.4f}")
print("\n" + "=" * 60)
print("CLASSIFICATION REPORT")
print("=" * 60)
print(classification_report(y_test, y_pred, target_names=['Остался (0)', 'Ушёл (1)']))


### **Кратко о метриках**

**Ключевые метрики для оценки модели:**

- **Accuracy** — доля правильных предсказаний (может быть обманчивой при несбалансированных данных)
- **ROC-AUC** — показывает, насколько хорошо модель различает классы (0.5 = плохо, >0.8 = хорошо)
- **Precision** — из всех предсказанных "уйдет", сколько действительно уйдут
- **Recall** — из всех реально ушедших, сколько мы нашли

<div style="padding: 12px 16px; margin: 16px 0; border-radius: 6px; border-left: 4px solid #3b82f6; background-color: #eff6ff; color: #1e40af;">
<strong>ℹ️ Для нашей задачи:</strong><br>ROC-AUC > 0.8 говорит о хорошем качестве модели. Высокий Recall важен — лучше найти больше потенциально уходящих клиентов.
</div>


### **Визаулизация**

Глаза нам даны не просто так - смотрим на картинки.

In [ ]:
# Визуализация результатов
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0], cbar=True)
axes[0].set_title('Матрица ошибок (Confusion Matrix)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Предсказанное значение', fontsize=12)
axes[0].set_ylabel('Истинное значение', fontsize=12)
axes[0].set_xticklabels(['Остался', 'Ушёл'])
axes[0].set_yticklabels(['Остался', 'Ушёл'])

# ROC Curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
axes[1].plot(fpr, tpr, color='#e74c3c', lw=2, label=f'ROC curve (AUC = {roc_auc:.3f})')
axes[1].plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--', label='Random classifier')
axes[1].set_xlabel('False Positive Rate', fontsize=12)
axes[1].set_ylabel('True Positive Rate', fontsize=12)
axes[1].set_title('ROC Кривая', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()


- `Confusion Matrix` показывает, сколько предсказаний были правильными/неправильными
- `ROC-кривая` показывает `trade-off` между` True Positive Rate` и `False Positive Rate`

#### **Посмотрим на веса**

In [ ]:
# Feature Importance - какие признаки наиболее важны для модели?
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print("Топ-10 самых важных признаков:")
print(feature_importance.head(10).to_string(index=False))

In [ ]:
# Визуализация

plt.figure(figsize=(12, 6))
top_features = feature_importance.head(15)
plt.barh(range(len(top_features)), top_features['importance'], color='#3498db')
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Важность признака', fontsize=12)
plt.title('Топ-15 важных признаков для модели', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

**Вопросы на подумать:**
- Какие признаки оказались самыми важными?
- Имеет ли это смысл с бизнес-точки зрения?
- Как можно использовать эту информацию?

---

### **Шаг 2: `Pickle` - `Pickle` - `Pickle` (🥒)**

<div style="padding: 12px 16px; margin: 16px 0; border-radius: 6px; border-left: 4px solid #3b82f6; background-color: #eff6ff; color: #1e40af;">
<strong>💡 О production!</strong><br>В production мы НЕ будем переобучать модель при каждом запросе. Нужно сохранить модель и уметь восстановить её без повторного обучения.
</div>

#### **Что такое `Pickle`?**

**`Pickle`** — это стандартный модуль Python для сериализации объектов. Он превращает любой Python-объект (модель, список, словарь и т.д.) в поток байтов, который можно сохранить в файл или передать по сети, а затем восстановить обратно.

**Как это работает:**
1. **`pickle.dump()`** — сериализует объект и записывает его в файл
2. **`pickle.load()`** — читает файл и десериализует объект обратно

#### **Ключевые параметры и методы**

**`pickle.dump(obj, file, protocol=None, fix_imports=True)`:**
- `obj` — объект для сохранения (наша модель)
- `file` — файловый объект, открытый в бинарном режиме ('wb')
- `protocol` — версия протокола (от 0 до 5, по умолчанию самый новый)
- `fix_imports` — автоматически исправлять импорты при загрузке (по умолчанию True)

**`pickle.load(file, fix_imports=True, encoding='ASCII')`:**
- `file` — файловый объект, открытый в бинарном режиме ('rb')
- `fix_imports` — исправлять импорты при загрузке
- `encoding` — кодировка для старых протоколов

---

#### **Что сохраняет `Pickle`?**

<div style="padding: 12px 16px; margin: 16px 0; border-radius: 6px; border-left: 4px solid #f59e0b; background-color: #fffbf0; color: #92400e;">
<strong>⚠️ Важно понимать!</strong><br>Pickle сохраняет <strong>состояние объекта</strong> (атрибуты, параметры, обученные веса модели), но <strong>НЕ сам класс</strong>. Класс должен быть доступен при загрузке (например, CatBoost должен быть установлен).
</div>

**Что сохраняется:**
- ✅ Все атрибуты объекта (например, `model.feature_names_`, `model.feature_importances_`)
- ✅ Обученные веса модели
- ✅ Параметры модели
- ✅ Вложенные объекты (если они поддерживают `pickle`)

**Что НЕ сохраняется:**
- ❌ Сам класс и его методы — класс должен быть доступен при загрузке
- ❌ Исходный код методов класса
- ❌ Метаданные о том, где находится определение класса
- ❌ Внешние зависимости (другие модули, которые использует класс)

---

#### **Безопасность Pickle**
<div style="padding: 12px 16px; margin: 16px 0; border-radius: 6px; border-left: 4px solid #ef4444; background-color: #fef2f2; color: #dc2626;"><strong>🚨 ВАЖНО: НИКОГДА не загружайте pickle-файлы из ненадежных источников!</strong></div>

**Уязвимости:**
- Выполнение произвольного кода при десериализации
- Атаки на целостность системы
- Внедрение вредоносного кода

**Меры безопасности:**
- ✅ Загружайте только свои файлы
- ✅ Используйте хеши для проверки целостности
- ✅ Ограничивайте источники загрузки
- ❌ НЕ загружайте файлы от неизвестных отправителей
- ❌ НЕ используйте pickle для межсерверной коммуникации

#### **Практический пример: Сохраняем и загружаем модель**

In [ ]:
# Создаем директорию для моделей и импортируем pickle
import os
import pickle

model_dir = 'models'
os.makedirs(model_dir, exist_ok=True)

print(f"📁 Директория '{model_dir}' создана или уже существует")

In [ ]:
# Сохранение модели через Pickle
print("💾 Сохраняем модель через Pickle...")
print("-" * 60)

# Путь к файлу модели
model_path = os.path.join(model_dir, 'churn_model.pkl')

# ВАЖНО: открываем файл в бинарном режиме ('wb' = write binary)
# Это обязательно для pickle!

with open(model_path, 'wb') as f:
    # pickle.dump() сериализует объект модели и записывает в файл
    # protocol=HIGHEST_PROTOCOL - берет самый актуальный протокол
    
    pickle.dump(model, f, protocol=pickle.HIGHEST_PROTOCOL)

print(f"✅ Модель сохранена: {model_path}")
print(f"   Размер файла: {os.path.getsize(model_path) / 1024:.2f} KB")
print(f"   Количество признаков: {len(model.feature_names_)}")
print("-" * 60)
print("🎉 Модель успешно сохранена через Pickle!")

# Также сохраним список признаков отдельно для удобства
feature_names_path = os.path.join(model_dir, 'feature_names.pkl')
with open(feature_names_path, 'wb') as f:
    pickle.dump(model.feature_names_, f, protocol=4)
print(f"✅ Список признаков сохранен: {feature_names_path}")


### **Проверка: загружаем и тестируем модель**

<div style="padding: 12px 16px; margin: 16px 0; border-radius: 6px; border-left: 4px solid #f59e0b; background-color: #fffbf0; color: #92400e;">
<strong>⚠️ Важно проверить!</strong><br>Убедитесь, что модель корректно загружается и работает перед деплоем.
</div>

In [ ]:
# Проверка: загрузка и тестирование модели через Pickle
print("🔍 Проверяем загрузку модели через Pickle...\n")

# Загружаем модель: ВАЖНО - открываем файл в бинарном режиме ('rb' = read binary)
with open(model_path, 'rb') as f:
    loaded_model = pickle.load(f)

# Загружаем список признаков
with open(feature_names_path, 'rb') as f:
    loaded_features = pickle.load(f)

print("✅ Модель успешно загружена через Pickle!")
print(f"\nТип модели: {type(loaded_model).__name__}")
print(f"Количество признаков: {len(loaded_features)}\n")

print(f"Список признаков: ")

print(loaded_features)

In [ ]:
# Тестируем на одном примере
test_sample = X_test.iloc[0:1]

prediction = loaded_model.predict(test_sample)
prediction_proba = loaded_model.predict_proba(test_sample)

print()
print("=" * 60)
print("ТЕСТ ПРЕДСКАЗАНИЯ")
print("=" * 60)

print(f"\nИстинное значение: {'Уйдёт' if y_test.iloc[0] == 1 else 'Останется'}")
print(f"Предсказание: {'Уйдёт' if prediction[0] == 1 else 'Останется'}\n")
print(f"Вероятность остаться: {prediction_proba[0][0]:.3%}")
print(f"Вероятность уйти: {prediction_proba[0][1]:.3%}")

#### **Сравнение с альтернативами**

<div style="padding: 12px 16px; margin: 16px 0; border-radius: 6px; border-left: 4px solid #3b82f6; background-color: #eff6ff; color: #1e40af;">
<strong>ℹ️ Существуют альтернативы pickle</strong><br> Но мы сегодня будем использовать его, он универсальный, не требует дополнительных библиотек и достаточен для наших задач.
</div>


| Способ | Преимущества | Недостатки | Когда использовать |
|--------|--------------|------------|-------------------|
| **Pickle (.pkl)** | ✅ Стандартная библиотека Python (не нужно устанавливать)<br>✅ Универсальный (работает с любыми Python объектами)<br>✅ Простой в использовании | ⚠️ Большой размер файлов (может быть в 2-3 раза больше)<br>⚠️ Вопросы безопасности (не загружать из ненадежных источников) | Эксперименты, универсальные решения, быстрый старт |
| **Joblib (.joblib)** | ✅ **Значительно меньше размер файлов** (оптимизирован для больших NumPy массивов)<br>✅ **Быстрее работает** с sklearn моделями<br>✅ **Параллельная обработка** больших данных | ❌ Нужно устанавливать (`pip install joblib`)<br>❌ Менее универсальный (лучше работает с NumPy/sklearn) | Production ML-модели, большие модели с NumPy массивами |


#### **📝 Помощь в ДЗ: Сохранение LinearRegression**

<div style="padding: 12px 16px; margin: 16px 0; border-radius: 6px; border-left: 4px solid #10b981; background-color: #f0fdf4; color: #047857;">
<strong>💡 Для домашнего задания:</strong><br>В вашем ДЗ вы наверняка будете использовать LinearRegression, вот краткий пример сохранения через Pickle.
</div>

In [ ]:
# Для дз, чтобы можно было скопировать пример:
# Не запускать тут, это для примера
 
from sklearn.linear_model import LinearRegression
import pickle
import os

# Тут обучаем
# model = LinearRegression()
# model.fit(X_train, y_train)

# Сохранение модели
model_dir = 'models'
os.makedirs(model_dir, exist_ok=True)

model_path = os.path.join(model_dir, 'linear_model.pkl')
with open(model_path, 'wb') as f:
    pickle.dump(model, f, protocol=pickle.HIGHEST_PROTOCOL)

print(f"✅ Модель сохранена: {model_path}")

# Загрузка модели
with open(model_path, 'rb') as f:
    loaded_model = pickle.load(f)

# Использование
prediction = loaded_model.predict(X_new)

---

## **🔹 Часть 2: Создание приложения в Streamlit**

<div style="padding: 12px 16px; margin: 16px 0; border-radius: 6px; border-left: 4px solid #3b82f6; background-color: #eff6ff; color: #1e40af;">
<strong>ℹ️ Что такое Streamlit?</strong><br>Streamlit — это Python библиотека для быстрого создания веб-приложений. Пишем на чистом Python, без HTML/CSS/JS!
</div>

### **Обзор основных элементов Streamlit**

Streamlit предоставляет множество готовых компонентов для создания интерактивных интерфейсов. Вот основные элементы, которые мы используем:

#### **📝 Текст и заголовки**

```python
st.title("🎯 Главный заголовок")           # Самый большой заголовок
st.header("Заголовок раздела")             # Заголовок раздела
st.subheader("Подзаголовок")               # Подзаголовок
st.write("Обычный текст")                  # Произвольный текст
st.markdown("**Жирный** или *курсив*")    # Markdown форматирование
st.caption("Мелкий текст-подпись")         # Мелкий текст
```

#### **📊 Отображение данных**

```python
st.dataframe(df)                           # Интерактивная таблица
st.table(df.head())                        # Статическая таблица
st.metric("Метрика", 42, "+5")            # Отображение метрики с изменением
```

#### **🎛️ Интерактивные виджеты**

```python
# Ввод данных
st.text_input("Введите текст")            # Текстовое поле
st.number_input("Число", min_value=0)     # Числовое поле
st.selectbox("Выберите", ["A", "B", "C"]) # Выпадающий список
st.multiselect("Выберите несколько", [...]) # Множественный выбор
st.slider("Слайдер", 0, 100, 50)         # Ползунок
st.checkbox("Согласен")                    # Чекбокс
st.radio("Выберите один", ["A", "B"])     # Радио-кнопки

# Файлы
st.file_uploader("Загрузить файл")        # Загрузка файла
st.download_button("Скачать", data)       # Кнопка скачивания

# Действия
st.button("Нажми меня")                   # Кнопка
```

#### **📈 Визуализации**

```python
st.plotly_chart(fig)                      # Интерактивный график Plotly
st.pyplot(fig)                            # График Matplotlib
st.line_chart(df)                         # Линейный график
st.bar_chart(df)                          # Столбчатая диаграмма
st.map(df)                                # Карта с маркерами
```

#### **🎨 Компоновка**

```python
col1, col2, col3 = st.columns(3)          # Создание колонок
with col1:
    st.write("Колонка 1")

st.sidebar.button("Меню")                 # Боковая панель
with st.expander("Развернуть"):           # Сворачиваемый блок
    st.write("Скрытый контент")

tab1, tab2 = st.tabs(["Вкладка 1", "Вкладка 2"])  # Вкладки
with tab1:
    st.write("Содержимое вкладки")
```

#### **⚡ Оптимизация производительности**

```python
@st.cache_resource                        # Кэширование объектов (модели, данные)
def load_model():
    return pickle.load(...)

@st.cache_data                            # Кэширование данных
def load_data():
    return pd.read_csv(...)
```

#### **🔄 Обработка состояний**

```python
if st.button("Клик"):
    st.session_state['key'] = 'value'      # Сохранение в сессии
    st.rerun()                             # Перезагрузка приложения
```

<div style="padding: 12px 16px; margin: 16px 0; border-radius: 6px; border-left: 4px solid #10b981; background-color: #f0fdf4; color: #047857;">
<strong>💡 Это основные элементы Streamlit:</strong><br> В нашем приложении мы используем: `st.title()`, `st.file_uploader()`, `st.dataframe()`, `st.plotly_chart()`, `st.metric()`, `st.columns()` и другие. Все элементы автоматически обновляют интерфейс при взаимодействии!
</div>


### **Запуск приложения!**

**Одна простая команда** - `streamlit run app_name.py`!

Открыть приложение можно будет по ссылке, которая выведется:

```bash
You can now view your Streamlit app in your browser.

Local URL: http://localhost:8501
Network URL: http://192.168.1.37:8501
```

_**P.S.** Порт может быть другим_

### **Структура app.py**

Перейдем к нашему приложению!
Создаем файл `app.py` в корне проекта. Разберём ключевые компоненты приложения:


**Ключевые компоненты приложения:**

#### **1. Загрузка модели через Pickle**

```python
import pickle
import streamlit as st

@st.cache_resource  # Кэшируем модель (загружается только один раз)
def load_model():
    with open('models/churn_model.pkl', 'rb') as f:
        model = pickle.load(f)
    with open('models/feature_names.pkl', 'rb') as f:
        feature_names = pickle.load(f)
    return model, feature_names
```

<div style="padding: 12px 16px; margin: 16px 0; border-radius: 6px; border-left: 4px solid #3b82f6; background-color: #eff6ff; color: #1e40af;">
<strong>ℹ️ @st.cache_resource</strong><br>Этот декоратор кэширует модель — она загрузится только один раз при первом запуске, а не при каждом обновлении страницы!
</div>

#### **2. Загрузка CSV файла и обработка данных**

```python
@st.cache_data  # Кэшируем загруженные данные
def load_data(uploaded_file):
    return pd.read_csv(uploaded_file)

# В интерфейсе:
uploaded_file = st.file_uploader("Загрузите CSV", type=["csv"])
if uploaded_file:
    df = load_data(uploaded_file)
```

#### **3. Интерактивные виджеты для ввода данных**

```python
# Выпадающий список
state = st.selectbox("Штат", ["NY", "CA", "TX"])

# Слайдер для числовых значений
account_length = st.slider("Длина аккаунта (месяцы)", 0, 100, 50)

# Чекбокс
international_plan = st.checkbox("Международный план")
```

#### **4. Визуализация результатов**

```python
# Метрика (показывает ключевой показатель)
st.metric("Вероятность оттока", "45%", delta="-5%")

# Прогресс-бар для визуализации вероятности
probability = 0.45
st.progress_bar(probability, text=f"{probability*100:.0f}%")

# Графики с Plotly (для красивых интерактивных визуализаций)
import plotly.express as px
fig = px.pie(df, names='churn_label')
st.plotly_chart(fig)
```

---

# **Часть 3: Разбор приложения app.py**

Теперь давайте разберём готовое приложение для предсказания оттока клиентов. Мы посмотрим, как все изученные виджеты работают вместе в реальном проекте.

## **Структура app.py**

### 1️⃣ Конфигурация страницы

```python
st.set_page_config(
    page_title="Churn Prediction",
    page_icon="🎯",
    layout="wide",
    initial_sidebar_state="expanded"
)
```

> 💡 Комментарий: задаём глобальные настройки страницы один раз; удобно описывать название, иконку и поведение боковой панели.

### 2️⃣ Кэширование модели

```python
from catboost import CatBoostClassifier

@st.cache_resource
def load_model():
    model = CatBoostClassifier()
    model.load_model('models/churn_model.pkl')
    feature_names = model.feature_names_
    return model, feature_names
```

> 💡 Комментарий: загружаем модель ровно один раз и сразу получаем список признаков из файла `.pkl`.

```python
def prepare_features(df, feature_names):
    """Приводим данные к формату обучения модели"""
    df_proc = df.copy()
    # Преобразуем категориальные признаки в строки (как при обучении)
    for col in feature_names:
        if col in df_proc.columns:
            if df_proc[col].dtype in ('object', 'bool'):
                df_proc[col] = df_proc[col].astype(str)
    return df_proc[feature_names]
```

**3. Основной интерфейс приложения:**

```python
# Загружаем модель
model, feature_names = load_model()

# Загрузка файла
uploaded_file = st.file_uploader("Загрузите CSV", type=["csv"])

if uploaded_file:
    df = pd.read_csv(uploaded_file)
    # Подготовка данных
    features = prepare_features(df, feature_names)
    # Предсказание
    probabilities = model.predict_proba(features)[:, 1]
    predictions = (probabilities >= 0.5).astype(int)
    
    # Визуализация результатов
    st.metric("Вероятность оттока", f"{probabilities[0]*100:.1f}%")
    st.progress_bar(probabilities[0])
```

<div style="padding: 12px 16px; margin: 16px 0; border-radius: 6px; border-left: 4px solid #10b981; background-color: #f0fdf4; color: #047857;">
<strong>✅ Готово!</strong><br>Теперь у вас есть рабочее приложение! Полный код находится в файле <code>app.py</code>.
</div>


---

# **Часть 4. Деплой на Streamlit**

## **Что понадобится?**

<div style="padding: 12px 16px; margin: 16px 0; border-radius: 6px; border-left: 4px solid #ef4444; background-color: #fef2f2; color: #dc2626;">
<strong>🚨 ВАЖНО: Версия Python!</strong><br>Для успешного деплоя необходим <strong>Python 3.11</strong>. Это актуальная и рекомендуемая версия для Streamlit.
</div>

Для деплоя на Streamlit Cloud нужно:

1. **GitHub репозиторий** с вашим кодом
2. **requirements.txt** со всеми зависимостями
3. **Python 3.11** (нужно указать при деплое)
4. **Папка models/** с сохранённой моделью (`.pkl`)
5. **app.py** - ваше приложение

### **Примерная структура проекта:**

```
churn-prediction-app/
│
├── app.py                     # Основное приложение
├── requirements.txt           # Зависимости Python
├── README.md                  # Описание проекта
├── .gitignore                 # Игнорируемые файлы
│
├── models/                    # Сохранённые модели
│   ├── churn_model.pkl        # CatBoost модель
│   └── feature_names.pkl      # Список признаков
```

## **Как задеплоить?**

### **Шаг 1: Создайте GitHub репозиторий**

<div style="padding: 12px 16px; margin: 16px 0; border-radius: 6px; border-left: 4px solid #f59e0b; background-color: #fffbf0; color: #92400e;">
<strong>⚠️ Важно:</strong><br>Репозиторий должен быть <strong>Public</strong> для бесплатного деплоя на Streamlit Cloud!
</div>

1. Создайте новый репозиторий на GitHub (Public)
2. Загрузите туда ваш код:
   - `app.py`
   - `requirements.txt`
   - Папка `models/` с моделью

### **Шаг 2: Деплой на Streamlit Cloud**

1. Перейдите на [streamlit.io/cloud](https://streamlit.io/cloud)
2. Войдите через GitHub
3. Нажмите "New app"
4. Выберите ваш репозиторий
5. Укажите путь к `app.py`
6. Нажмите "Deploy"

<div style="padding: 12px 16px; margin: 16px 0; border-radius: 6px; border-left: 4px solid #10b981; background-color: #f0fdf4; color: #047857;">
<strong>✅ Готово!</strong><br>Ваше приложение будет доступно по ссылке вида: <code>https://YOUR_APP_NAME.streamlit.app</code>
</div>

**Перейдите в папку с проектом**
```bash
cd <Ваша папка>
```

**Инициализируйте git**
```bash
git init
```

**Добавьте все файлы**
```bash
git add .
```

((Создайте первый коммит))
```bash
git commit -m "Initial commit: Churn prediction app"
```

**Подключите удалённый репозиторий**
```bash
git branch -M main
git remote add origin https://github.com/YOUR_USERNAME/churn-prediction-app.git
```
**Отправьте код на GitHub**
```bash
git push -u origin main
```

_Замените `YOUR_USERNAME` на ваш GitHub username!_

#### **Шаг 3. Проверьте, что все файлы загружены**
- Зайдите на GitHub и убедитесь, что файлы на месте
- Особенно проверьте папку `models/` с моделями!

### **Шаг 4: Регистрация на Streamlit Cloud**

#### **4.1 Создайте аккаунт**
1. Перейдите на [share.streamlit.io](https://share.streamlit.io/)
2. Нажмите **"Sign up"**
3. Выберите **"Continue with GitHub"**
4. Авторизуйтесь через GitHub

#### **4.2 Разрешите доступ к репозиториям**
1. Streamlit попросит доступ к вашим репозиториям
2. Нажмите **"Authorize streamlit"**
3. Выберите, какие репозитории разрешить (можно все или конкретные)

### **Шаг 5: Создание нового приложения**

#### **5.1 Запуск деплоя**
1. На главной странице Streamlit Cloud нажмите **"New app"**
2. Заполните форму:
   - **Repository:** выберите `YOUR_USERNAME/churn-prediction-app`
   - **Branch:** `main`
   - **Main file path:** `app.py`
   - **App URL** (необязательно): придумайте короткий URL, например `churn-predictor`

#### **5.2 Дополнительные настройки**

<div style="padding: 12px 16px; margin: 16px 0; border-radius: 6px; border-left: 4px solid #f59e0b; background-color: #fffbf0; color: #92400e;">
<strong>⚠️ Обязательно!</strong><br>В разделе <strong>"Advanced settings"</strong> → <strong>"Python version"</strong> выберите <strong>Python 3.11</strong>! Это критически важно для совместимости со всеми библиотеками.
</div>

- **Advanced settings:**
  - **Python version: выберите `3.11`** (это актуальная версия для Streamlit)
  - **Secrets:** если нужны API ключи (не нужно для нашего проекта)

#### **5.3 Нажмите `"Deploy!"`**

### **Шаг 6: Ожидание деплоя**

#### **Что происходит?**
1. ✅ Streamlit клонирует ваш репозиторий
2. ✅ Устанавливает зависимости из `requirements.txt`
3. ✅ Загружает модели
4. ✅ Запускает приложение

**Время ожидания:** 2-5 минут

#### **Что вы увидите?**
- Логи установки пакетов
- Прогресс-бар
- Если всё успешно - ваше приложение!

#### **После приложение будет готово по сслыке:**
```
https://YOUR_APP_NAME.streamlit.app
```

### **Дополнительно: Обновление приложения**

#### **Обновляем код**

```bash
# Внесите изменения в код
# Например, добавьте новую функцию в app.py

# Сделайте коммит
git add .
git commit -m "Добавлена новая функция"

# Отправьте на GitHub
git push
```

#### **Обновление самого Streamlit**
- В вашем веб-приложении (или в кабинете) нажимаете `"Manage app"`, там будут логи и прочие настройки
- Вам нужна кнопка `«Прочее»` (или три точки) → `"Reboot app"`

---

## **Типичные проблемы при деплое и их решения**

### **❌ Проблема 1: Отсутствуют библиотеки**

**Ошибка:** `ModuleNotFoundError: No module named 'plotly'` или ошибки с версиями

**Решение:** Добавьте все библиотеки в `requirements.txt` и сделайте `git push`. Используйте совместимые версии или `>=` вместо `==`.

### **❌ Проблема 2: Модель не найдена**

**Ошибка:** `FileNotFoundError: models/churn_model.pkl`

**Решение:** Убедитесь, что папка `models/` добавлена в git: `git add models/ && git commit -m "Add models" && git push`

### **❌ Проблема 3: Медленная работа или падения**

**Решение:** 
- Используйте `@st.cache_resource` для кэширования модели
- Добавьте `try-except` блоки для обработки ошибок
- Проверьте логи в Streamlit Cloud

### **Чек-лист:**

- [ ] Всё работает локально на **Python 3.11**
- [ ] **Python 3.11** указана в настройках деплоя Streamlit Cloud
- [ ] `requirements.txt` содержит все библиотеки
- [ ] Модели добавлены в git (`churn_model.pkl`, `feature_names.pkl`)
- [ ] Используется кэширование (`@st.cache_resource`)
- [ ] Репозиторий Public (для бесплатного деплоя)


---

# **📖 Полезное**

## **Официальная документация**

### **Streamlit**
- [Streamlit Documentation](https://docs.streamlit.io/) - полная документация
- [Streamlit Gallery](https://streamlit.io/gallery) - примеры приложений
- [Streamlit YouTube](https://www.youtube.com/@streamlitofficial) - официальный канал
- [30 Days of Streamlit](https://30days.streamlit.app/) - 30-дневный челлендж

### **Machine Learning**
- [Scikit-learn Documentation](https://scikit-learn.org/stable/)
- [Pandas Documentation](https://pandas.pydata.org/docs/)
- [Plotly Python](https://plotly.com/python/)

